In [ ]:
!pip install mlxtend

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules, apriori
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [14]:
df = pd.read_csv('/tmdb_5000_movies.csv')
df = df.drop(df.loc[df['genres'] == '[]'].index) # drop rows with empty genres
df = df.loc[:, ['original_title', 'genres']]

In [15]:
import re
# parse genre and convert it to python list
def split_genres(x):
  r = x[1:-1].split('}, ')
  genres = []
  pattern = re.compile(r'(\"name\": )(\"[\d\D]*\")')
  for genre in r:
    genres.append(re.search(pattern, genre).group(2))
  
  return genres


In [19]:
data = list(df['genres'].apply(split_genres))
data[:5] # genre has been parsed and ready to be encode

[['"Action"', '"Adventure"', '"Fantasy"', '"Science Fiction"'],
 ['"Adventure"', '"Fantasy"', '"Action"'],
 ['"Action"', '"Adventure"', '"Crime"'],
 ['"Action"', '"Crime"', '"Drama"', '"Thriller"'],
 ['"Action"', '"Adventure"', '"Science Fiction"']]

In [20]:
te = TransactionEncoder()
te_data = te.fit(data).transform(data) # transform data into group of genres
df = pd.DataFrame(te_data,columns=te.columns_)
df.head()

,"""Action""","""Adventure""","""Animation""","""Comedy""","""Crime""","""Documentary""","""Drama""","""Family""","""Fantasy""","""Foreign""","""History""","""Horror""","""Music""","""Mystery""","""Romance""","""Science Fiction""","""TV Movie""","""Thriller""","""War""","""Western"""
0,True,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False
1,True,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False
4,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False


In [ ]:
df1 = apriori(df, min_support=0.03, use_colnames=True) # find frequent value of genres using apriori algorithm

In [23]:
rules = association_rules(df1, metric="lift", min_threshold=1) # create association rules
rules.loc[rules['lift'] == rules['lift'].max()] # select rules with the highest lift

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
17,"(""Family"")","(""Animation"")",0.107435,0.049005,0.040838,0.380117,7.75666,0.035573,1.534152
